In [ ]:
from pyspark.sql import Window
from pyspark.sql.types import * 
from pyspark.sql.functions import * 

In [ ]:
configs = {
    "fs.azure.account.auth.type": "OAuth",
    "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
    "fs.azure.account.oauth2.client.id": "",
    "fs.azure.account.oauth2.client.secret": "",
    "fs.azure.account.oauth2.client.endpoint": ""
}


##mount raw-data

In [ ]:
# container@storageacc
dbutils.fs.mount(
    source = "abfss://raw-data@myprojects.dfs.core.windows.net",
    mount_point = "/mnt/raw-election-data",
    extra_configs = configs
)

     

True

In [ ]:
dbutils.fs.ls("/mnt/raw-election-data")

[FileInfo(path='dbfs:/mnt/raw-election-data/2014/', name='2014/', size=0, modificationTime=1717484336000),
 FileInfo(path='dbfs:/mnt/raw-election-data/2019/', name='2019/', size=0, modificationTime=1717484963000)]

In [ ]:
dbutils.fs.ls("/mnt/raw-election-data/2014/")

[FileInfo(path='dbfs:/mnt/raw-election-data/2014/results.csv', name='results.csv', size=705846, modificationTime=1717497446000)]

##load dim-states dimension table form transformed-data folder

In [ ]:
# container@storageacc
dbutils.fs.mount(
    source = "abfss://transformed-data@myprojects.dfs.core.windows.net",
    mount_point = "/mnt/dim-electiondata",
    extra_configs = configs
)

True

In [ ]:
dbutils.fs.ls("/mnt/dim-electiondata")

[FileInfo(path='dbfs:/mnt/dim-electiondata/dimension-tables/', name='dimension-tables/', size=0, modificationTime=1717485955000)]

In [ ]:
dbutils.fs.ls("/mnt/dim-electiondata/dimension-tables/")

[FileInfo(path='dbfs:/mnt/dim-electiondata/dimension-tables/dim_candidate_2014/', name='dim_candidate_2014/', size=0, modificationTime=1717582243000),
 FileInfo(path='dbfs:/mnt/dim-electiondata/dimension-tables/dim_states/', name='dim_states/', size=0, modificationTime=1717582306000),
 FileInfo(path='dbfs:/mnt/dim-electiondata/dimension-tables/dim_states_codes.csv', name='dim_states_codes.csv', size=539, modificationTime=1717582487000)]

In [ ]:
spark

##Load state_codes data

In [ ]:
dim_states_df = spark.read.format("csv")\
.option("inferschema","true")\
.option("header","true")\
.option("mode","failfast")\
.load("dbfs:/mnt/dim-electiondata/dimension-tables/dim_states_codes.csv")


In [ ]:
dim_states_df.show()

+--------------------+------------+
|          state_name|abbreviation|
+--------------------+------------+
|Andaman & Nicobar...|          AN|
|      Andhra Pradesh|          AP|
|   Arunachal Pradesh|          AR|
|               Assam|          AS|
|               Bihar|          BR|
|          Chandigarh|          CH|
|        Chhattisgarh|          CT|
|Dadra & Nagar Haveli|          DN|
|         Daman & Diu|          DD|
|        NCT OF Delhi|          DL|
|                 Goa|          GA|
|             Gujarat|          GJ|
|             Haryana|          HR|
|    Himachal Pradesh|          HP|
|     Jammu & Kashmir|          JK|
|           Jharkhand|          JH|
|           Karnataka|          KA|
|              Kerala|          KL|
|         Lakshadweep|          LD|
|      Madhya Pradesh|          MP|
+--------------------+------------+
only showing top 20 rows



In [ ]:
dim_states_df.count()

36

##Load 2014 results data

In [ ]:
raw_2014_results_df = spark.read.format("csv")\
.option("inferschema","true")\
.option("header","true")\
.option("mode","failfast")\
.load("dbfs:/mnt/raw-election-data/2014/results.csv")


In [ ]:
raw_2014_results_df.show(3)

+--------------+---------+-------------+---+----+--------+-----+------------+-------------+------------+-----------+--------------+
|         state|  pc_name|    candidate|sex| age|category|party|party_symbol|general_votes|postal_votes|total_votes|total_electors|
+--------------+---------+-------------+---+----+--------+-----+------------+-------------+------------+-----------+--------------+
|Andhra Pradesh|Adilabad | GODAM NAGESH|  M|49.0|      ST|  TRS|         Car|       425762|        5085|     430847|       1386282|
|Andhra Pradesh|Adilabad |       NARESH|  M|37.0|      ST|  INC|        Hand|       257994|        1563|     259557|       1386282|
|Andhra Pradesh|Adilabad |RAMESH RATHOD|  M|48.0|      ST|  TDP|     Bicycle|       182879|        1319|     184198|       1386282|
+--------------+---------+-------------+---+----+--------+-----+------------+-------------+------------+-----------+--------------+
only showing top 3 rows



In [ ]:
raw_2014_results_df.count()

8355

In [ ]:
raw_2019_results_df.printSchema()

root
 |-- state: string (nullable = true)
 |-- pc_name: string (nullable = true)
 |-- candidate: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: double (nullable = true)
 |-- category: string (nullable = true)
 |-- party: string (nullable = true)
 |-- party_symbol: string (nullable = true)
 |-- general_votes: integer (nullable = true)
 |-- postal_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- total_electors: integer (nullable = true)



##Load 2019 results data

In [ ]:
raw_2019_results_df = spark.read.format("csv")\
.option("inferschema","true")\
.option("header","true")\
.option("mode","failfast")\
.load("dbfs:/mnt/raw-election-data/2019/results.csv")

In [ ]:
raw_2019_results_df.show(3)

+--------------+-------+--------------------+------+----+--------+-----+------------+-------------+------------+-----------+--------------+
|         state|pc_name|           candidate|   sex| age|category|party|party_symbol|general_votes|postal_votes|total_votes|total_electors|
+--------------+-------+--------------------+------+----+--------+-----+------------+-------------+------------+-----------+--------------+
|Andhra Pradesh| Aruku | KISHORE CHANDRA DEO|  MALE|72.0|      ST|  TDP|     Bicycle|       336163|        1938|     338101|       1451418|
|Andhra Pradesh| Aruku |Dr. KOSURI KASI V...|  MALE|54.0|      ST|  BJP|       Lotus|        17578|         289|      17867|       1451418|
|Andhra Pradesh| Aruku |    GODDETI. MADHAVI|FEMALE|26.0|      ST|YSRCP| Ceiling Fan|       557561|        4629|     562190|       1451418|
+--------------+-------+--------------------+------+----+--------+-----+------------+-------------+------------+-----------+--------------+
only showing top 3 r

In [ ]:
raw_2019_results_df.printSchema()

root
 |-- state: string (nullable = true)
 |-- pc_name: string (nullable = true)
 |-- candidate: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: double (nullable = true)
 |-- category: string (nullable = true)
 |-- party: string (nullable = true)
 |-- party_symbol: string (nullable = true)
 |-- general_votes: integer (nullable = true)
 |-- postal_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- total_electors: integer (nullable = true)



In [ ]:
raw_2019_results_df.count()

8597

##Data Cleaning

1. Data might contain constituencies spelling mismatches and some constituencies
may be listed with identical names. Proper validation is required.

Approach:- Standardizing  Names (remove extra spaces, convert to lower case)

In [ ]:

def standardize_data(df):
    for column in df.columns:
        df = df.withColumn(column, trim(lower(col(column))))
    return df

raw_2014_results_df = standardize_data(raw_2014_results_df)
raw_2019_results_df = standardize_data(raw_2019_results_df) 
dim_states_df = standardize_data(dim_states_df)



In [ ]:
raw_2014_results_df.show()

+--------------+----------+--------------------+----+----+--------+-----+--------------+-------------+------------+-----------+--------------+
|         state|   pc_name|           candidate| sex| age|category|party|  party_symbol|general_votes|postal_votes|total_votes|total_electors|
+--------------+----------+--------------------+----+----+--------+-----+--------------+-------------+------------+-----------+--------------+
|andhra pradesh|  adilabad|        godam nagesh|   m|49.0|      st|  trs|           car|       425762|        5085|     430847|       1386282|
|andhra pradesh|  adilabad|              naresh|   m|37.0|      st|  inc|          hand|       257994|        1563|     259557|       1386282|
|andhra pradesh|  adilabad|       ramesh rathod|   m|48.0|      st|  tdp|       bicycle|       182879|        1319|     184198|       1386282|
|andhra pradesh|  adilabad|     rathod sadashiv|   m|55.0|      st|  bsp|      elephant|        94363|          57|      94420|       1386282|

In [ ]:
raw_2019_results_df.show()

+--------------+----------+--------------------+------+----+--------+-----+-------------+-------------+------------+-----------+--------------+
|         state|   pc_name|           candidate|   sex| age|category|party| party_symbol|general_votes|postal_votes|total_votes|total_electors|
+--------------+----------+--------------------+------+----+--------+-----+-------------+-------------+------------+-----------+--------------+
|andhra pradesh|     aruku| kishore chandra deo|  male|72.0|      st|  tdp|      bicycle|       336163|        1938|     338101|       1451418|
|andhra pradesh|     aruku|dr. kosuri kasi v...|  male|54.0|      st|  bjp|        lotus|        17578|         289|      17867|       1451418|
|andhra pradesh|     aruku|    goddeti. madhavi|female|26.0|      st|ysrcp|  ceiling fan|       557561|        4629|     562190|       1451418|
|andhra pradesh|     aruku|shruti devi vyric...|female|46.0|      st|  inc|         hand|        17656|          74|      17730|       1

In [ ]:
dim_states_df.show(truncate=False)

+-------------------------+------------+
|state_name               |abbreviation|
+-------------------------+------------+
|andaman & nicobar islands|an          |
|andhra pradesh           |ap          |
|arunachal pradesh        |ar          |
|assam                    |as          |
|bihar                    |br          |
|chandigarh               |ch          |
|chhattisgarh             |ct          |
|dadra & nagar haveli     |dn          |
|daman & diu              |dd          |
|nct of delhi             |dl          |
|goa                      |ga          |
|gujarat                  |gj          |
|haryana                  |hr          |
|himachal pradesh         |hp          |
|jammu & kashmir          |jk          |
|jharkhand                |jh          |
|karnataka                |ka          |
|kerala                   |kl          |
|lakshadweep              |ld          |
|madhya pradesh           |mp          |
+-------------------------+------------+
only showing top

2. In 2014, Andhra Pradesh underwent bifurcation. For simplicity, all constituencies
from that year should be attributed to Telangana state. This includes constituencies
such as Adilabad, Hyderabad, Warangal, etc., which should be considered part of
Telangana ratherthan Andhra Pradesh forthe year 2014.

Approach Step 1:- 

2019 dataset has telangana in dataset so from 2019 dataset ill collect all telangana related constituences into a list 



In [ ]:
telangana_constituencies_df= raw_2019_results_df.filter(col("state")=="telangana").groupBy("pc_name").agg(count("*"))\
.select(col("pc_name")) 
telangana_constituencies_df.show()

+------------+
|     pc_name|
+------------+
|     bhongir|
|   nizamabad|
|nagarkurnool|
|  peddapalle|
|    chevella|
| mahbubnagar|
|   hyderabad|
|    nalgonda|
|  malkajgiri|
|   zahirabad|
|    warangal|
|  karimnagar|
|       medak|
|    adilabad|
| mahabubabad|
|     khammam|
| secundrabad|
+------------+



In [ ]:
telangana_constituencies_df.count()

17

In [ ]:
telangana_constituencies_list = [row.pc_name for row in telangana_constituencies_df.collect()]

In [ ]:
telangana_constituencies_list 

['bhongir',
 'nizamabad',
 'nagarkurnool',
 'peddapalle',
 'chevella',
 'mahbubnagar',
 'hyderabad',
 'nalgonda',
 'malkajgiri',
 'zahirabad',
 'warangal',
 'karimnagar',
 'medak',
 'adilabad',
 'mahabubabad',
 'khammam',
 'secundrabad']

Approach Step2:-

in 2014 dataset in state of Andra pradesh where ever this telangana related constituences comes in ill resplace state name to telangana 

In [ ]:
raw_2014_results_df = raw_2014_results_df.withColumn(
    "state",
    when((col("state") == "andhra pradesh") & (col("pc_name").isin(telangana_constituencies_list)), "telangana")
    .otherwise(col("state"))
)

Approach validation

In [ ]:
raw_2014_results_df.filter(col("state")=="telangana").groupBy("pc_name").agg(count("*"))\
.select(col("pc_name")).show()


+------------+
|     pc_name|
+------------+
|     bhongir|
|   nizamabad|
|nagarkurnool|
|  peddapalle|
| mahbubnagar|
|   hyderabad|
|    nalgonda|
|  malkajgiri|
|   zahirabad|
|    warangal|
|  karimnagar|
|       medak|
|    adilabad|
| mahabubabad|
|     khammam|
| secundrabad|
+------------+



In [ ]:
raw_2014_results_df.filter(col("state")=="telangana").show()

+---------+----------+--------------------+----+----+--------+-----+--------------+-------------+------------+-----------+--------------+
|    state|   pc_name|           candidate| sex| age|category|party|  party_symbol|general_votes|postal_votes|total_votes|total_electors|
+---------+----------+--------------------+----+----+--------+-----+--------------+-------------+------------+-----------+--------------+
|telangana|  adilabad|        godam nagesh|   m|49.0|      st|  trs|           car|       425762|        5085|     430847|       1386282|
|telangana|  adilabad|              naresh|   m|37.0|      st|  inc|          hand|       257994|        1563|     259557|       1386282|
|telangana|  adilabad|       ramesh rathod|   m|48.0|      st|  tdp|       bicycle|       182879|        1319|     184198|       1386282|
|telangana|  adilabad|     rathod sadashiv|   m|55.0|      st|  bsp|      elephant|        94363|          57|      94420|       1386282|
|telangana|  adilabad|    nethawat

##Data Transformation

  dim_states creation 
- state_id (Primary Key)
- state_name (String)
- abbreviation (String)

In [ ]:
dim_states_df.printSchema()

root
 |-- state_name: string (nullable = true)
 |-- abbreviation: string (nullable = true)



In [ ]:
dim_states_df.count()

36

In [ ]:
window_spec = Window.partitionBy().orderBy("state_name")
dim_states_with_id_df = dim_states_df.withColumn("state_id", row_number().over(window_spec))

In [ ]:
dim_states_transformed_df = dim_states_with_id_df.select("state_id", "state_name", "abbreviation")

In [ ]:
dim_states_transformed_df.show(36)

+--------+--------------------+------------+
|state_id|          state_name|abbreviation|
+--------+--------------------+------------+
|       1|andaman & nicobar...|          an|
|       2|      andhra pradesh|          ap|
|       3|   arunachal pradesh|          ar|
|       4|               assam|          as|
|       5|               bihar|          br|
|       6|          chandigarh|          ch|
|       7|        chhattisgarh|          ct|
|       8|dadra & nagar haveli|          dn|
|       9|         daman & diu|          dd|
|      10|                 goa|          ga|
|      11|             gujarat|          gj|
|      12|             haryana|          hr|
|      13|    himachal pradesh|          hp|
|      14|     jammu & kashmir|          jk|
|      15|           jharkhand|          jh|
|      16|           karnataka|          ka|
|      17|              kerala|          kl|
|      18|         lakshadweep|          ld|
|      19|      madhya pradesh|          mp|
|      20|

In [ ]:
dim_states_transformed_df.write.format("csv") \
.option("header", "true") \
.option("path", "dbfs:/mnt/dim-electiondata/dimension-tables/dim_states") \
.save()

In [ ]:
dbutils.fs.ls("dbfs:/mnt/dim-electiondata/dimension-tables/dim_states")

[FileInfo(path='dbfs:/mnt/dim-electiondata/dimension-tables/dim_states/_SUCCESS', name='_SUCCESS', size=0, modificationTime=1717582306000),
 FileInfo(path='dbfs:/mnt/dim-electiondata/dimension-tables/dim_states/_committed_2841320491649554479', name='_committed_2841320491649554479', size=113, modificationTime=1717582306000),
 FileInfo(path='dbfs:/mnt/dim-electiondata/dimension-tables/dim_states/_started_2841320491649554479', name='_started_2841320491649554479', size=0, modificationTime=1717582306000),
 FileInfo(path='dbfs:/mnt/dim-electiondata/dimension-tables/dim_states/part-00000-tid-2841320491649554479-1c11fdd8-1834-4b27-b752-e581e1650487-245-1-c000.csv', name='part-00000-tid-2841320491649554479-1c11fdd8-1834-4b27-b752-e581e1650487-245-1-c000.csv', size=644, modificationTime=1717582306000)]

In [ ]:
raw_2014_results_df.printSchema()

root
 |-- state: string (nullable = true)
 |-- pc_name: string (nullable = true)
 |-- candidate: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: string (nullable = true)
 |-- category: string (nullable = true)
 |-- party: string (nullable = true)
 |-- party_symbol: string (nullable = true)
 |-- general_votes: string (nullable = true)
 |-- postal_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- total_electors: string (nullable = true)



**All data types are string which is not correct hence casting appropriate data**

In [ ]:
transformed_2014_data_df = raw_2014_results_df.withColumn("age", col("age").cast("integer")) \
                                         .withColumn("general_votes", col("general_votes").cast("long")) \
                                         .withColumn("postal_votes", col("postal_votes").cast("long")) \
                                         .withColumn("total_votes", col("total_votes").cast("long")) \
                                         .withColumn("total_electors", col("total_electors").cast("long"))

In [ ]:
transformed_2014_data_df.printSchema()

root
 |-- state: string (nullable = true)
 |-- pc_name: string (nullable = true)
 |-- candidate: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- category: string (nullable = true)
 |-- party: string (nullable = true)
 |-- party_symbol: string (nullable = true)
 |-- general_votes: long (nullable = true)
 |-- postal_votes: long (nullable = true)
 |-- total_votes: long (nullable = true)
 |-- total_electors: long (nullable = true)



**doing the same thing for 2019 data**

In [ ]:
transformed_2019_data_df = raw_2019_results_df.withColumn("age", col("age").cast("integer")) \
                                         .withColumn("general_votes", col("general_votes").cast("long")) \
                                         .withColumn("postal_votes", col("postal_votes").cast("long")) \
                                         .withColumn("total_votes", col("total_votes").cast("long")) \
                                         .withColumn("total_electors", col("total_electors").cast("long"))

In [ ]:
transformed_2019_data_df.printSchema()

root
 |-- state: string (nullable = true)
 |-- pc_name: string (nullable = true)
 |-- candidate: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- category: string (nullable = true)
 |-- party: string (nullable = true)
 |-- party_symbol: string (nullable = true)
 |-- general_votes: long (nullable = true)
 |-- postal_votes: long (nullable = true)
 |-- total_votes: long (nullable = true)
 |-- total_electors: long (nullable = true)



In [ ]:
%fs refreshMounts

Mounts successfully refreshed.
res6: Boolean = true

Upon looking up data in excel

**Problem** :- 

the electorate has pressed None of above (NOTA) for the party 
In our dataset row party value is recorded as Nota , candidate value as None of Above or in some data points as NOTA , sex value is empty , age value is empty , category value is empty 
party_symbol is empty 



In [ ]:
#Normalize NOTA values and fill missing values for NOTA rows 

def normalize_nota_values(df):
    df = df.withColumn(
        "candidate", 
        when(col("candidate").isin(["none of the above", "nota"]), "nota").otherwise(col("candidate"))
    ).withColumn(
        "sex", 
        when(col("candidate") == "nota", lit("na")).otherwise(col("sex"))
    ).withColumn(
        "age", 
        when(col("candidate") == "nota", lit(-1)).otherwise(col("age"))
    ).withColumn(
        "category", 
        when(col("candidate") == "nota", lit("na")).otherwise(col("category"))
    ).withColumn(
        "party_symbol", 
        when(col("candidate") == "nota", lit("na")).otherwise(col("party_symbol"))
    )
    return df 

transformed_2014_data_df = normalize_nota_values(transformed_2014_data_df) 
transformed_2019_data_df = normalize_nota_values(transformed_2019_data_df) 


In [ ]:
transformed_2014_data_df.select("candidate", "sex", "age", "category").filter(
    (col("candidate").isNull()) |
    (col("sex").isNull()) |
    (col("age").isNull()) |
    (col("category").isNull())
).show()

+---------+---+---+--------+
|candidate|sex|age|category|
+---------+---+---+--------+
+---------+---+---+--------+



In [ ]:
transformed_2014_data_df.select("candidate", "sex", "age", "category").filter(col("candidate")=="nota").show()

+---------+---+---+--------+
|candidate|sex|age|category|
+---------+---+---+--------+
|     nota| na| -1|      na|
|     nota| na| -1|      na|
|     nota| na| -1|      na|
|     nota| na| -1|      na|
|     nota| na| -1|      na|
|     nota| na| -1|      na|
|     nota| na| -1|      na|
|     nota| na| -1|      na|
|     nota| na| -1|      na|
|     nota| na| -1|      na|
|     nota| na| -1|      na|
|     nota| na| -1|      na|
|     nota| na| -1|      na|
|     nota| na| -1|      na|
|     nota| na| -1|      na|
|     nota| na| -1|      na|
|     nota| na| -1|      na|
|     nota| na| -1|      na|
|     nota| na| -1|      na|
|     nota| na| -1|      na|
+---------+---+---+--------+
only showing top 20 rows



In [ ]:
transformed_2019_data_df.select("candidate", "sex", "age", "category").filter(
    (col("candidate").isNull()) |
    (col("sex").isNull()) |
    (col("age").isNull()) |
    (col("category").isNull())
).show()

+---------+---+---+--------+
|candidate|sex|age|category|
+---------+---+---+--------+
+---------+---+---+--------+



**Creating Dimension Tables**

**dim_candidate_2014**

In [ ]:

candidate_df = transformed_2014_data_df.select("candidate", "sex", "age", "category").distinct()

window_spec = Window.partitionBy().orderBy("candidate")

dim_candidate_2014 = candidate_df.withColumn("candidate_id", row_number().over(window_spec))

dim_candidate_2014 = dim_candidate_2014.select("candidate_id", "candidate", "sex", "age", "category")


In [ ]:
dim_candidate_2014.count()

7829

In [ ]:
dim_candidate_2014.show(5,truncate=False)

+------------+---------------------------+---+---+--------+
|candidate_id|candidate                  |sex|age|category|
+------------+---------------------------+---+---+--------+
|1           |"""comando"" kamal kishor" |m  |62 |sc      |
|2           |"""sandeep soni"""         |m  |36 |gen     |
|3           |"""yogesh nanaji samrite"""|m  |42 |gen     |
|4           |"ajay shah ""makrai"""     |m  |58 |st      |
|5           |"ashok shukla "" senani""" |m  |43 |gen     |
+------------+---------------------------+---+---+--------+
only showing top 5 rows



In [ ]:
dim_candidate_2014.printSchema()

root
 |-- candidate_id: integer (nullable = false)
 |-- candidate: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- category: string (nullable = true)



In [ ]:
dim_candidate_2014.write.format("csv")\
.option("header","true")\
.mode("overwrite")\
.save("dbfs:/mnt/dim-electiondata/dimension-tables/dim_candidates_2014")

In [ ]:
dbutils.fs.ls("dbfs:/mnt/dim-electiondata/dimension-tables/dim_candidates_2014")

[FileInfo(path='dbfs:/mnt/dim-electiondata/dimension-tables/dim_candidates_2014/_SUCCESS', name='_SUCCESS', size=0, modificationTime=1717681315000),
 FileInfo(path='dbfs:/mnt/dim-electiondata/dimension-tables/dim_candidates_2014/_committed_3506348337970583856', name='_committed_3506348337970583856', size=213, modificationTime=1717681315000),
 FileInfo(path='dbfs:/mnt/dim-electiondata/dimension-tables/dim_candidates_2014/_committed_6499819068117468041', name='_committed_6499819068117468041', size=113, modificationTime=1717586001000),
 FileInfo(path='dbfs:/mnt/dim-electiondata/dimension-tables/dim_candidates_2014/_committed_vacuum8432399382141231816', name='_committed_vacuum8432399382141231816', size=96, modificationTime=1717681315000),
 FileInfo(path='dbfs:/mnt/dim-electiondata/dimension-tables/dim_candidates_2014/_started_3506348337970583856', name='_started_3506348337970583856', size=0, modificationTime=1717681315000),
 FileInfo(path='dbfs:/mnt/dim-electiondata/dimension-tables/dim_ca

**dim_candidate_2019**

In [ ]:
candidate_df = transformed_2019_data_df.select("candidate", "sex", "age", "category").distinct()

window_spec = Window.partitionBy().orderBy("candidate")

dim_candidate_2019 = candidate_df.withColumn("candidate_id", row_number().over(window_spec))

dim_candidate_2019 = dim_candidate_2019.select("candidate_id", "candidate", "sex", "age", "category")

In [ ]:
dim_candidate_2019.show(5,truncate=False)

+------------+--------------------------------+----+---+--------+
|candidate_id|candidate                       |sex |age|category|
+------------+--------------------------------+----+---+--------+
|1           |"badrilal ""akela"""            |male|43 |sc      |
|2           |"chandra bhadra singh ""ssonu"""|male|43 |general |
|3           |"charan singh ""babrik"""       |male|50 |sc      |
|4           |"deepak kumar ""nanhe"""        |male|32 |sc      |
|5           |"devendra jain "" milan"""      |male|73 |general |
+------------+--------------------------------+----+---+--------+
only showing top 5 rows



In [ ]:
dim_candidate_2019.count()

8026

In [ ]:
dim_candidate_2019.write.format("csv")\
.option("header", "true")\
.mode("overwrite")\
.save("dbfs:/mnt/dim-electiondata/dimension-tables/dim_candidates_2019")


In [ ]:
dbutils.fs.ls("dbfs:/mnt/dim-electiondata/dimension-tables/dim_candidates_2019")

[FileInfo(path='dbfs:/mnt/dim-electiondata/dimension-tables/dim_candidates_2019/_committed_4252852686347576550', name='_committed_4252852686347576550', size=202, modificationTime=1717681382000),
 FileInfo(path='dbfs:/mnt/dim-electiondata/dimension-tables/dim_candidates_2019/_committed_4271809794576177511', name='_committed_4271809794576177511', size=212, modificationTime=1717670942000),
 FileInfo(path='dbfs:/mnt/dim-electiondata/dimension-tables/dim_candidates_2019/_started_4252852686347576550', name='_started_4252852686347576550', size=0, modificationTime=1717681382000),
 FileInfo(path='dbfs:/mnt/dim-electiondata/dimension-tables/dim_candidates_2019/part-00000-tid-4252852686347576550-6dbcf119-fa0f-422c-b7c5-f01a3e603a6b-1020-1-c000.csv', name='part-00000-tid-4252852686347576550-6dbcf119-fa0f-422c-b7c5-f01a3e603a6b-1020-1-c000.csv', size=299399, modificationTime=1717681382000)]

**dim_parties_2014**

In [ ]:
party_df = transformed_2014_data_df.select("party", "party_symbol").distinct()

windowSpec = Window.orderBy("party")

dim_party_2014 = party_df.withColumn("party_id", row_number().over(windowSpec))


dim_party_2014 = dim_party_2014.select("party_id", "party", "party_symbol")

In [ ]:
dim_party_2014.count()

785

In [ ]:
dim_party_2014.write.format("csv")\
.option("header","true")\
.mode("overwrite")\
.save("dbfs:/mnt/dim-electiondata/dimension-tables/dim_party_2014")

In [ ]:
dbutils.fs.ls("dbfs:/mnt/dim-electiondata/dimension-tables/dim_party_2014")

[FileInfo(path='dbfs:/mnt/dim-electiondata/dimension-tables/dim_party_2014/_SUCCESS', name='_SUCCESS', size=0, modificationTime=1717681429000),
 FileInfo(path='dbfs:/mnt/dim-electiondata/dimension-tables/dim_party_2014/_committed_2818562097415332561', name='_committed_2818562097415332561', size=113, modificationTime=1717586985000),
 FileInfo(path='dbfs:/mnt/dim-electiondata/dimension-tables/dim_party_2014/_committed_3741891464046765812', name='_committed_3741891464046765812', size=213, modificationTime=1717681429000),
 FileInfo(path='dbfs:/mnt/dim-electiondata/dimension-tables/dim_party_2014/_committed_vacuum369667157037208071', name='_committed_vacuum369667157037208071', size=95, modificationTime=1717681429000),
 FileInfo(path='dbfs:/mnt/dim-electiondata/dimension-tables/dim_party_2014/_started_3741891464046765812', name='_started_3741891464046765812', size=0, modificationTime=1717681429000),
 FileInfo(path='dbfs:/mnt/dim-electiondata/dimension-tables/dim_party_2014/part-00000-tid-374

**dim_parties_2019**

In [ ]:
window_spec = Window.orderBy("party") 

party_df = transformed_2019_data_df.select("party","party_symbol").distinct() 

dim_party_2019 = party_df.withColumn("party_id",row_number().over(window_spec)) 

dim_party_2019.select("party_id","party","party_symbol")

DataFrame[party_id: int, party: string, party_symbol: string]

In [ ]:
dim_party_2019.count()

1146

In [ ]:
dim_party_2019.show()

+------+--------------------+--------+
| party|        party_symbol|party_id|
+------+--------------------+--------+
|  aaap|               broom|       1|
|aabhap|             balloon|       2|
|  aacp|             batsman|       3|
|aahpty|               apple|       4|
|   aam|      chapati roller|       5|
|  aapp|       battery torch|       6|
|   abd|            neck tie|       7|
|  abep|   man blowing turha|       8|
|  abgp|                 saw|       9|
|  abgp|       battery torch|      10|
|  abhm|plate containing ...|      11|
|  abhm|      auto- rickshaw|      12|
|  abhm|             diamond|      13|
|  abhp|           ear rings|      14|
|  abhs|        gas cylinder|      15|
|  abjs|         ganna kisan|      16|
|  abjs|          helicopter|      17|
|  abjs|                 key|      18|
|  abjs|tractor chalata k...|      19|
|  abjs|       battery torch|      20|
+------+--------------------+--------+
only showing top 20 rows



In [ ]:
dim_party_2019.write.format("csv")\
.option("header","true")\
.mode("overwrite")\
.save("dbfs:/mnt/dim-electiondata/dimension-tables/dim_party_2019")

In [ ]:
dbutils.fs.ls("dbfs:/mnt/dim-electiondata/dimension-tables/dim_party_2019")

[FileInfo(path='dbfs:/mnt/dim-electiondata/dimension-tables/dim_party_2019/_SUCCESS', name='_SUCCESS', size=0, modificationTime=1717681476000),
 FileInfo(path='dbfs:/mnt/dim-electiondata/dimension-tables/dim_party_2019/_committed_1996756012143222330', name='_committed_1996756012143222330', size=113, modificationTime=1717586907000),
 FileInfo(path='dbfs:/mnt/dim-electiondata/dimension-tables/dim_party_2019/_committed_2394674530473628009', name='_committed_2394674530473628009', size=213, modificationTime=1717681476000),
 FileInfo(path='dbfs:/mnt/dim-electiondata/dimension-tables/dim_party_2019/_committed_vacuum3404610880866159259', name='_committed_vacuum3404610880866159259', size=96, modificationTime=1717681476000),
 FileInfo(path='dbfs:/mnt/dim-electiondata/dimension-tables/dim_party_2019/_started_2394674530473628009', name='_started_2394674530473628009', size=0, modificationTime=1717681475000),
 FileInfo(path='dbfs:/mnt/dim-electiondata/dimension-tables/dim_party_2019/part-00000-tid-2

**Load all news formed Dimension tables**

In [ ]:
states_dimension_df = spark.read.format("csv")\
.option("header","true")\
.option("inferschema","true")\
.load("dbfs:/mnt/dim-electiondata/dimension-tables/dim_states")

In [ ]:
states_dimension_df.show(2)

+--------+--------------------+------------+
|state_id|          state_name|abbreviation|
+--------+--------------------+------------+
|       1|andaman & nicobar...|          an|
|       2|      andhra pradesh|          ap|
+--------+--------------------+------------+
only showing top 2 rows



In [ ]:
candidates_2014_dimension_df = spark.read.format("csv")\
.option("header","true")\
.option("inferschema","true")\
.load("dbfs:/mnt/dim-electiondata/dimension-tables/dim_candidates_2014")

In [ ]:
candidates_2014_dimension_df.show(2)

+------------+--------------------+---+---+--------+
|candidate_id|           candidate|sex|age|category|
+------------+--------------------+---+---+--------+
|           1|"""comando"" kama...|  m| 62|      sc|
|           2|  """sandeep soni"""|  m| 36|     gen|
+------------+--------------------+---+---+--------+
only showing top 2 rows



In [ ]:
candidates_2019_dimension_df = spark.read.format("csv")\
.option("header","true")\
.option("inferschema","true")\
.load("dbfs:/mnt/dim-electiondata/dimension-tables/dim_candidates_2019")

In [ ]:
candidates_2019_dimension_df.show(2)

+------------+--------------------+----+---+--------+
|candidate_id|           candidate| sex|age|category|
+------------+--------------------+----+---+--------+
|           1|"badrilal ""akela"""|male| 43|      sc|
|           2|"chandra bhadra s...|male| 43| general|
+------------+--------------------+----+---+--------+
only showing top 2 rows



In [ ]:
parties_2014_dimension_df = spark.read.format("csv")\
.option("header","true")\
.option("inferschema","true")\
.load("dbfs:/mnt/dim-electiondata/dimension-tables/dim_party_2014")

In [ ]:
parties_2014_dimension_df.show(2)

+--------+------+-------------+
|party_id| party| party_symbol|
+--------+------+-------------+
|       1| a s p|walking stick|
|       2|aa s p| gas cylinder|
+--------+------+-------------+
only showing top 2 rows



In [ ]:
parties_2019_dimension_df = spark.read.format("csv")\
.option("header","true")\
.option("inferschema","true")\
.load("dbfs:/mnt/dim-electiondata/dimension-tables/dim_party_2019")

In [ ]:
parties_2019_dimension_df.show(2)

+------+------------+--------+
| party|party_symbol|party_id|
+------+------------+--------+
|  aaap|       broom|       1|
|aabhap|     balloon|       2|
+------+------------+--------+
only showing top 2 rows



**Creating Fact Tables**

**fact_2014**

In [ ]:
transformed_2014_data_df.count()

8355

join states_dimension_df to get state_id

join parties_2014_dimension_df to get party id

join with candidates_2014_dimension_df to get candidate_id

In [ ]:
join_with_states_dim_df = transformed_2014_data_df.join(states_dimension_df, 
                                                transformed_2014_data_df["state"] == states_dimension_df["state_name"], 
                                                "inner").select(transformed_2014_data_df["*"],states_dimension_df["state_id"])

In [ ]:
join_with_states_dim_df.count()

8355

In [ ]:
join_with_states_party_dim_df = join_with_states_dim_df.join(parties_2014_dimension_df,
                             
                             (join_with_states_dim_df["party"]==parties_2014_dimension_df["party"]) & 

                             (join_with_states_dim_df["party_symbol"]==parties_2014_dimension_df["party_symbol"]),
                             
                             "inner").select(parties_2014_dimension_df["party_id"],join_with_states_dim_df["*"])

In [ ]:
join_with_states_party_dim_df.show(1)

+--------+---------+--------+------------+---+---+--------+-----+------------+-------------+------------+-----------+--------------+--------+
|party_id|    state| pc_name|   candidate|sex|age|category|party|party_symbol|general_votes|postal_votes|total_votes|total_electors|state_id|
+--------+---------+--------+------------+---+---+--------+-----+------------+-------------+------------+-----------+--------------+--------+
|     760|telangana|adilabad|godam nagesh|  m| 49|      st|  trs|         car|       425762|        5085|     430847|       1386282|      32|
+--------+---------+--------+------------+---+---+--------+-----+------------+-------------+------------+-----------+--------------+--------+
only showing top 1 row



In [ ]:
join_with_states_party_dim_df.count()

8355

In [ ]:
join_with_candidate_state_party_df = join_with_states_party_dim_df.join(
    candidates_2014_dimension_df,
    (join_with_states_party_dim_df["candidate"] == candidates_2014_dimension_df["candidate"]) &
    (join_with_states_party_dim_df["sex"] == candidates_2014_dimension_df["sex"]) &
    (join_with_states_party_dim_df["age"] == candidates_2014_dimension_df["age"]) &
    (join_with_states_party_dim_df["category"] == candidates_2014_dimension_df["category"]),
    "inner"
).select(
    candidates_2014_dimension_df["candidate_id"].alias("candidate_id"),
    join_with_states_party_dim_df["*"]
)

In [ ]:
join_with_candidate_state_party_df.count()

8355

In [ ]:
join_with_candidate_state_party_df.printSchema()

root
 |-- candidate_id: integer (nullable = true)
 |-- party_id: integer (nullable = true)
 |-- state: string (nullable = true)
 |-- pc_name: string (nullable = true)
 |-- candidate: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- category: string (nullable = true)
 |-- party: string (nullable = true)
 |-- party_symbol: string (nullable = true)
 |-- general_votes: long (nullable = true)
 |-- postal_votes: long (nullable = true)
 |-- total_votes: long (nullable = true)
 |-- total_electors: long (nullable = true)
 |-- state_id: integer (nullable = true)



In [ ]:
fact_2014 = join_with_candidate_state_party_df.select(
"state_id",
"pc_name",
"candidate_id",
"party_id",
"general_votes",
"postal_votes",
"total_votes",
"total_electors"
)

In [ ]:
fact_2014.show(1)

+--------+--------+------------+--------+-------------+------------+-----------+--------------+
|state_id| pc_name|candidate_id|party_id|general_votes|postal_votes|total_votes|total_electors|
+--------+--------+------------+--------+-------------+------------+-----------+--------------+
|      32|adilabad|        2263|     760|       425762|        5085|     430847|       1386282|
+--------+--------+------------+--------+-------------+------------+-----------+--------------+
only showing top 1 row



In [ ]:
len(fact_2014.columns)

8

In [ ]:
fact_2014.write.format("csv")\
.option("header","true")\
.mode("overwrite")\
.save("dbfs:/mnt/dim-electiondata/fact-tables/fact_2014")

In [ ]:
dbutils.fs.ls("dbfs:/mnt/dim-electiondata/fact-tables/fact_2014")

[FileInfo(path='dbfs:/mnt/dim-electiondata/fact-tables/fact_2014/_SUCCESS', name='_SUCCESS', size=0, modificationTime=1717768732000),
 FileInfo(path='dbfs:/mnt/dim-electiondata/fact-tables/fact_2014/_committed_5628462581743197371', name='_committed_5628462581743197371', size=114, modificationTime=1717768732000),
 FileInfo(path='dbfs:/mnt/dim-electiondata/fact-tables/fact_2014/_started_5628462581743197371', name='_started_5628462581743197371', size=0, modificationTime=1717768732000),
 FileInfo(path='dbfs:/mnt/dim-electiondata/fact-tables/fact_2014/part-00000-tid-5628462581743197371-67e1591e-4185-45a6-ba09-0cdd7c1aafd7-1084-1-c000.csv', name='part-00000-tid-5628462581743197371-67e1591e-4185-45a6-ba09-0cdd7c1aafd7-1084-1-c000.csv', size=355710, modificationTime=1717768732000)]

**fact_2019**

In [ ]:
transformed_2019_data_df.count()

8597

In [ ]:
join_with_states_dim_df = transformed_2019_data_df.join(states_dimension_df, 
                                                transformed_2019_data_df["state"] == states_dimension_df["state_name"], 
                                                "inner").select(transformed_2019_data_df["*"],states_dimension_df["state_id"])

In [ ]:
join_with_states_party_dim_df = join_with_states_dim_df.join(parties_2019_dimension_df,
                             
                             (join_with_states_dim_df["party"]==parties_2019_dimension_df["party"]) & 

                             (join_with_states_dim_df["party_symbol"]==parties_2019_dimension_df["party_symbol"]),
                             
                             "inner").select(parties_2019_dimension_df["party_id"],join_with_states_dim_df["*"])

In [ ]:
join_with_candidate_state_party_df = join_with_states_party_dim_df.join(
    candidates_2019_dimension_df,
    (join_with_states_party_dim_df["candidate"] == candidates_2019_dimension_df["candidate"]) &
    (join_with_states_party_dim_df["sex"] == candidates_2019_dimension_df["sex"]) &
    (join_with_states_party_dim_df["age"] == candidates_2019_dimension_df["age"]) &
    (join_with_states_party_dim_df["category"] == candidates_2019_dimension_df["category"]),
    "inner"
).select(
    candidates_2019_dimension_df["candidate_id"].alias("candidate_id"),
    join_with_states_party_dim_df["*"]
)

In [ ]:
fact_2019 = join_with_candidate_state_party_df.select(
"state_id",
"pc_name",
"candidate_id",
"party_id",
"general_votes",
"postal_votes",
"total_votes",
"total_electors"
)

In [ ]:
fact_2019.show(1)

+--------+-------+------------+--------+-------------+------------+-----------+--------------+
|state_id|pc_name|candidate_id|party_id|general_votes|postal_votes|total_votes|total_electors|
+--------+-------+------------+--------+-------------+------------+-----------+--------------+
|       2|  aruku|        3412|    1086|       336163|        1938|     338101|       1451418|
+--------+-------+------------+--------+-------------+------------+-----------+--------------+
only showing top 1 row



In [ ]:
fact_2019.count()

8597

In [ ]:
fact_2019.write.format("csv")\
.option("header","true")\
.mode("overwrite")\
.save("dbfs:/mnt/dim-electiondata/fact-tables/fact_2019")

In [ ]:
fact_2019.printSchema()

root
 |-- state_id: integer (nullable = true)
 |-- pc_name: string (nullable = true)
 |-- candidate_id: integer (nullable = true)
 |-- party_id: integer (nullable = true)
 |-- general_votes: long (nullable = true)
 |-- postal_votes: long (nullable = true)
 |-- total_votes: long (nullable = true)
 |-- total_electors: long (nullable = true)



**Load the fact_tables**

In [ ]:
my_schema = StructType([
    StructField("state_id", IntegerType(), True),
    StructField("pc_name", StringType(), True),
    StructField("candidate_id", IntegerType(), True),
    StructField("party_id", IntegerType(), True),
    StructField("general_votes", LongType(), True),
    StructField("postal_votes", LongType(), True),
    StructField("total_votes", LongType(), True),
    StructField("total_electors", LongType(), True)
])

fact_2014_df = spark.read.format("csv")\
.option("header","true")\
.schema(my_schema)\
.load("dbfs:/mnt/dim-electiondata/fact-tables/fact_2014")

In [ ]:
fact_2014_df.printSchema()

root
 |-- state_id: integer (nullable = true)
 |-- pc_name: string (nullable = true)
 |-- candidate_id: integer (nullable = true)
 |-- party_id: integer (nullable = true)
 |-- general_votes: long (nullable = true)
 |-- postal_votes: long (nullable = true)
 |-- total_votes: long (nullable = true)
 |-- total_electors: long (nullable = true)



In [ ]:
my_schema = StructType([
    StructField("state_id", IntegerType(), True),
    StructField("pc_name", StringType(), True),
    StructField("candidate_id", IntegerType(), True),
    StructField("party_id", IntegerType(), True),
    StructField("general_votes", LongType(), True),
    StructField("postal_votes", LongType(), True),
    StructField("total_votes", LongType(), True),
    StructField("total_electors", LongType(), True)
])

fact_2019_df = spark.read.format("csv")\
.option("header","true")\
.schema(my_schema)\
.load("dbfs:/mnt/dim-electiondata/fact-tables/fact_2019")

In [ ]:
fact_2019_df.printSchema()

root
 |-- state_id: integer (nullable = true)
 |-- pc_name: string (nullable = true)
 |-- candidate_id: integer (nullable = true)
 |-- party_id: integer (nullable = true)
 |-- general_votes: long (nullable = true)
 |-- postal_votes: long (nullable = true)
 |-- total_votes: long (nullable = true)
 |-- total_electors: long (nullable = true)



prepare spark sql tables

In [ ]:
states_dimension_df.createOrReplaceTempView("dim_states_table") 

#2014

parties_2014_dimension_df.createOrReplaceTempView("dim_parties_2014_table") 
candidates_2014_dimension_df.createOrReplaceTempView("dim_candidates_2014_table") 

#2019 

parties_2019_dimension_df.createOrReplaceTempView("dim_parties_2019_table") 
candidates_2019_dimension_df.createOrReplaceTempView("dim_candidates_2019_table") 


In [ ]:
fact_2014_df.createOrReplaceTempView("fact_2014_table") 
fact_2019_df.createOrReplaceTempView("fact_2019_table") 


In [ ]:
# container@storageacc
dbutils.fs.mount(
    source = "abfss://analytical-results@myprojects.dfs.core.windows.net",
    mount_point = "/mnt/analytics/",
    extra_configs = configs
)

True

# Primary Analysis

# 1.List top 5 / bottom 5 constituencies of 2014 and 2019 in terms of voterturnout ratio


**_Voter Turnout Ratio = (Total Votes Cast / Total Electors)× 100%_**

**top 5 constituencies of 2014 based on voter turnout ratio**

In [ ]:

spark.sql("""
select pc_name as constituency_name ,round((sum(total_votes)/total_electors)*100,2) as voter_turnout_ratio
from fact_2014_table 
group by pc_name,total_electors
order by voter_turnout_ratio desc 
limit 5
""").show()

+-----------------+-------------------+
|constituency_name|voter_turnout_ratio|
+-----------------+-------------------+
|           dhubri|              88.35|
|         nagaland|              87.82|
|           tamluk|              87.59|
|        bishnupur|              86.72|
|      lakshadweep|              86.61|
+-----------------+-------------------+



In [ ]:
df = fact_2014_df \
    .groupBy("pc_name", "total_electors") \
    .agg(round((sum("total_votes") / col("total_electors")) * 100, 2).alias("voter_turnout_ratio")) \
    .orderBy(col("voter_turnout_ratio").desc()) \
    .limit(5)

result_df = df.select(col("pc_name").alias("constituency_name"), "voter_turnout_ratio") 

result_df.show()

+-----------------+-------------------+
|constituency_name|voter_turnout_ratio|
+-----------------+-------------------+
|           dhubri|              88.35|
|         nagaland|              87.82|
|           tamluk|              87.59|
|        bishnupur|              86.72|
|      lakshadweep|              86.61|
+-----------------+-------------------+



In [ ]:
result_df.write.format("csv")\
.option("header","true")\
.mode("overwrite")\
.save("dbfs:/mnt/analytics/2014/top_5_constituencies_based_on_voterturnout_ratio")

**bottom 5 constituencies of 2014 based on voter turnout ratio**

In [ ]:
result_df = fact_2014_df \
    .groupBy("pc_name", "total_electors") \
    .agg(round((sum("total_votes") / col("total_electors")) * 100, 2).alias("voter_turnout_ratio")) \
    .orderBy(col("voter_turnout_ratio").asc()) \
    .limit(5)

result_df = result_df.select(col("pc_name").alias("constituency_name"), "voter_turnout_ratio") 
result_df.show()

+-----------------+-------------------+
|constituency_name|voter_turnout_ratio|
+-----------------+-------------------+
|         srinagar|              25.86|
|         anantnag|              28.84|
|        baramulla|              39.13|
|           kalyan|              42.88|
|      patna sahib|              45.33|
+-----------------+-------------------+



In [ ]:
result_df.write.format("csv")\
.option("header","true")\
.mode("overwrite")\
.save("dbfs:/mnt/analytics/2014/bottom_5_constituencies_based_on_voterturnout_ratio")

> **top 5 constituencies of 2019 based on voter turnout ratio**

In [ ]:
result_df = fact_2019_df \
    .groupBy("pc_name", "total_electors") \
    .agg(round((sum("total_votes") / col("total_electors")) * 100, 2).alias("voter_turnout_ratio")) \
    .orderBy(col("voter_turnout_ratio").desc()) \
    .limit(5)

result_df = result_df.select(col("pc_name").alias("constituency_name"), "voter_turnout_ratio") 

result_df.show()

+-----------------+-------------------+
|constituency_name|voter_turnout_ratio|
+-----------------+-------------------+
|           dhubri|              90.66|
|        bishnupur|              87.31|
|          barpeta|              86.55|
|       jalpaiguri|              86.49|
|   arunachal east|              86.46|
+-----------------+-------------------+



In [ ]:
result_df.write.format("csv")\
.option("header","true")\
.mode("overwrite")\
.save("dbfs:/mnt/analytics/2019/top_5_constituencies_based_on_voterturnout_ratio")

> **bottom 5 constituencies of 2019 based on voter turnout ratio**

In [ ]:
result_df = fact_2019_df \
    .groupBy("pc_name", "total_electors") \
    .agg(round((sum("total_votes") / col("total_electors")) * 100, 2).alias("voter_turnout_ratio")) \
    .orderBy(col("voter_turnout_ratio").asc()) \
    .limit(5)

result_df = result_df.select(col("pc_name").alias("constituency_name"), "voter_turnout_ratio") 
result_df.show()

+-----------------+-------------------+
|constituency_name|voter_turnout_ratio|
+-----------------+-------------------+
|         anantnag|               8.94|
|         srinagar|              14.43|
|        baramulla|              34.57|
|        hyderabad|              44.84|
|           kalyan|              45.29|
+-----------------+-------------------+



In [ ]:
result_df.write.format("csv")\
.option("header","true")\
.mode("overwrite")\
.save("dbfs:/mnt/analytics/2019/bottom_5_constituencies_based_on_voterturnout_ratio")

# 2. List top 5 / bottom 5 states of 2014 and 2019 in terms of voterturnout ratio?

**top 5 states of 2014 in terms of voterturnout ratio**

In [ ]:
top_5_states_voter_turnout_ratio_df =spark.sql(""" 
with q1 as 
(
SELECT state_id,total_electors
FROM fact_2014_table 
GROUP BY state_id,total_electors 
),
state_wise_total_electors as 
( 
select state_id,sum(total_electors) 
as overall_total_electors
from q1 
group by state_id 
),
state_wise_total_votes as 
(
SELECT state_id, sum(total_votes) as sum_of_total_votes
FROM fact_2014_table 
GROUP BY state_id 
), 
data_joins as 
(
select state_wise_total_votes.*,state_wise_total_electors.overall_total_electors
from state_wise_total_electors 
inner join state_wise_total_votes 
on state_wise_total_electors.state_id = state_wise_total_votes.state_id 
),
voter_turnout_ratio as 
(
select state_id, 
round((sum_of_total_votes/overall_total_electors)*100,2) as voter_turnout_ratio
from data_joins 
order by voter_turnout_ratio desc 
limit 5 
)
select  dim_states_table.state_name,voter_turnout_ratio.voter_turnout_ratio
from voter_turnout_ratio 
inner join dim_states_table
on voter_turnout_ratio.state_id=dim_states_table.state_id 
order by voter_turnout_ratio.voter_turnout_ratio desc
""") 





In [ ]:
top_5_states_voter_turnout_ratio_df.show()

+--------------------+-------------------+
|          state_name|voter_turnout_ratio|
+--------------------+-------------------+
|            nagaland|              87.82|
|         lakshadweep|              86.61|
|             tripura|              84.72|
|dadra & nagar haveli|              84.07|
|              sikkim|              83.33|
+--------------------+-------------------+



In [ ]:
top_5_states_voter_turnout_ratio_df.write.format("csv")\
.option("header","true")\
.mode("overwrite")\
.save("dbfs:/mnt/analytics/2014/top_5_states_based_on_voterturnout_ratio")

**bottom 5 states of 2014 based on voter turnout ratio**

In [ ]:
bottom_5_states_voter_turnout_ratio_df =spark.sql(""" 
with q1 as 
(
SELECT state_id,total_electors
FROM fact_2014_table 
GROUP BY state_id,total_electors 
),
state_wise_total_electors as 
( 
select state_id,sum(total_electors) 
as overall_total_electors
from q1 
group by state_id 
),
state_wise_total_votes as 
(
SELECT state_id, sum(total_votes) as sum_of_total_votes
FROM fact_2014_table 
GROUP BY state_id 
), 
data_joins as 
(
select state_wise_total_votes.*,state_wise_total_electors.overall_total_electors
from state_wise_total_electors 
inner join state_wise_total_votes 
on state_wise_total_electors.state_id = state_wise_total_votes.state_id 
),
voter_turnout_ratio as 
(
select state_id, 
round((sum_of_total_votes/overall_total_electors)*100,2) as voter_turnout_ratio
from data_joins 
order by voter_turnout_ratio asc
limit 5 
)
select  dim_states_table.state_name,voter_turnout_ratio.voter_turnout_ratio
from voter_turnout_ratio 
inner join dim_states_table
on voter_turnout_ratio.state_id=dim_states_table.state_id 
order by voter_turnout_ratio.voter_turnout_ratio asc
""") 

In [ ]:
bottom_5_states_voter_turnout_ratio_df.show()

+---------------+-------------------+
|     state_name|voter_turnout_ratio|
+---------------+-------------------+
|jammu & kashmir|              49.66|
|          bihar|              56.25|
|  uttar pradesh|              58.42|
|    maharashtra|              60.29|
| madhya pradesh|              61.59|
+---------------+-------------------+



In [ ]:
bottom_5_states_voter_turnout_ratio_df.write.format("csv")\
.option("header","true")\
.mode("overwrite")\
.save("dbfs:/mnt/analytics/2014/bottom_5_states_based_on_voterturnout_ratio")

**top 5 states of 2019 based on voter turnout ratio**

In [ ]:
top_5_2019_states_voter_turnout_ratio_df =spark.sql(""" 
with q1 as 
(
SELECT state_id,total_electors
FROM fact_2019_table 
GROUP BY state_id,total_electors 
),
state_wise_total_electors as 
( 
select state_id,sum(total_electors) 
as overall_total_electors
from q1 
group by state_id 
),
state_wise_total_votes as 
(
SELECT state_id, sum(total_votes) as sum_of_total_votes
FROM fact_2019_table 
GROUP BY state_id 
), 
data_joins as 
(
select state_wise_total_votes.*,state_wise_total_electors.overall_total_electors
from state_wise_total_electors 
inner join state_wise_total_votes 
on state_wise_total_electors.state_id = state_wise_total_votes.state_id 
),
voter_turnout_ratio as 
(
select state_id, 
round((sum_of_total_votes/overall_total_electors)*100,2) as voter_turnout_ratio
from data_joins 
order by voter_turnout_ratio desc
limit 5 
)
select  dim_states_table.state_name,voter_turnout_ratio.voter_turnout_ratio
from voter_turnout_ratio 
inner join dim_states_table
on voter_turnout_ratio.state_id=dim_states_table.state_id 
order by voter_turnout_ratio.voter_turnout_ratio desc
""") 

In [ ]:
top_5_2019_states_voter_turnout_ratio_df.show()

+-----------+-------------------+
| state_name|voter_turnout_ratio|
+-----------+-------------------+
|lakshadweep|              85.18|
|   nagaland|              82.91|
|    manipur|              82.54|
|    tripura|              82.35|
|west bengal|              81.72|
+-----------+-------------------+



In [ ]:
top_5_2019_states_voter_turnout_ratio_df.write.format("csv")\
.option("header","true")\
.mode("overwrite")\
.save("dbfs:/mnt/analytics/2019/top_5_states_based_on_voterturnout_ratio")

**bottom 5 states of 2019 based on voter turnout ratio**

In [ ]:
bottom_5_2019_states_voter_turnout_ratio_df =spark.sql(""" 
with q1 as 
(
SELECT state_id,total_electors
FROM fact_2019_table 
GROUP BY state_id,total_electors 
),
state_wise_total_electors as 
( 
select state_id,sum(total_electors) 
as overall_total_electors
from q1 
group by state_id 
),
state_wise_total_votes as 
(
SELECT state_id, sum(total_votes) as sum_of_total_votes
FROM fact_2019_table 
GROUP BY state_id 
), 
data_joins as 
(
select state_wise_total_votes.*,state_wise_total_electors.overall_total_electors
from state_wise_total_electors 
inner join state_wise_total_votes 
on state_wise_total_electors.state_id = state_wise_total_votes.state_id 
),
voter_turnout_ratio as 
(
select state_id, 
round((sum_of_total_votes/overall_total_electors)*100,2) as voter_turnout_ratio
from data_joins 
order by voter_turnout_ratio asc
limit 5 
)
select  dim_states_table.state_name,voter_turnout_ratio.voter_turnout_ratio
from voter_turnout_ratio 
inner join dim_states_table
on voter_turnout_ratio.state_id=dim_states_table.state_id 
order by voter_turnout_ratio.voter_turnout_ratio asc
""") 

In [ ]:
bottom_5_2019_states_voter_turnout_ratio_df.show()

+---------------+-------------------+
|     state_name|voter_turnout_ratio|
+---------------+-------------------+
|jammu & kashmir|              44.84|
|          bihar|               57.3|
|  uttar pradesh|              59.18|
|   nct of delhi|              60.58|
|    maharashtra|              60.96|
+---------------+-------------------+



In [ ]:
bottom_5_2019_states_voter_turnout_ratio_df.write.format("csv")\
.option("header","true")\
.mode("overwrite")\
.save("dbfs:/mnt/analytics/2019/bottom_5_states_based_on_voterturnout_ratio")

# 3. Which constituencies have elected the same party for two consecutive elections,rank them by % of votes to that winning party in 2019

In [ ]:
res_df = spark.sql("""
with party_2014 as
(
   select fact_2014_table.pc_name,fact_2014_table.total_votes,fact_2014_table.total_electors,
   dim_parties_2014_table.party 
   from fact_2014_table 
   inner join dim_parties_2014_table 
   on dim_parties_2014_table.party_id = fact_2014_table.party_id

),
rank_party_2014 as 
(
   select *,
   rank() over(partition by pc_name order by total_votes desc) as rank_2014
   from party_2014 
),
winner_2014 as 
(
select pc_name,party from rank_party_2014 
where rank_2014=1 
),
party_2019 as
(
   select fact_2019_table.pc_name,fact_2019_table.total_votes,fact_2019_table.total_electors,
   dim_parties_2019_table.party 
   from fact_2019_table 
   inner join dim_parties_2019_table 
   on dim_parties_2019_table.party_id = fact_2019_table.party_id

),
rank_party_2019 as 
(
   select *,
   rank() over(partition by pc_name order by total_votes desc) as rank_2019
   from party_2019 
),
winner_2019 as 
(
select pc_name,party from rank_party_2019 
where rank_2019=1 
) ,
union_data as 
(
select pc_name,party
from winner_2014 
union all 
select pc_name,party
from winner_2019
),
unique_combination_count as 
(
select pc_name,party,count(1) as cnt
from union_data 
group by pc_name,party 
),
consecutive_winners as 
( 
select pc_name,party from unique_combination_count 
where cnt=2
),
winning_party_vote_metrics as 
(
    select pc_name,party,total_votes,
    sum(total_votes) over(partition by pc_name) as total_votes_per_constituency
    from party_2019 
),
winning_party_vote_percentage_2019 as 
(
select pc_name,party,
round((total_votes/total_votes_per_constituency)*100,2 ) 
as vote_percentage
from winning_party_vote_metrics
),
join_winners_to_their_winning_percentage as 
(
select consecutive_winners.pc_name,consecutive_winners.party,
winning_party_vote_percentage_2019.vote_percentage 
from consecutive_winners
inner join winning_party_vote_percentage_2019 
on consecutive_winners.pc_name = winning_party_vote_percentage_2019.pc_name and  consecutive_winners.party=winning_party_vote_percentage_2019.party  
),
add_row_num as 
(
select *,
rank() over(partition by pc_name  order by vote_percentage desc) as rn
from  join_winners_to_their_winning_percentage
),
pick_top_1 as
(
select pc_name,party,vote_percentage
from add_row_num 
where rn=1 
order by vote_percentage desc
)
select * from pick_top_1


""")


In [ ]:
res_df.show(330)

+--------------------+------+---------------+
|             pc_name| party|vote_percentage|
+--------------------+------+---------------+
|               surat|   bjp|          74.47|
|             navsari|   bjp|          74.37|
|            vadodara|   bjp|           72.3|
|              kangra|   bjp|          72.02|
|            bhilwara|   bjp|          71.59|
|        mumbai north|   bjp|           71.4|
|              karnal|   bjp|          70.08|
|         gandhinagar|   bjp|          69.67|
|           rajsamand|   bjp|          69.61|
|         hoshangabad|   bjp|          69.35|
|           faridabad|   bjp|           68.8|
|               mandi|   bjp|          68.75|
|             garhwal|   bjp|          68.25|
|             vidisha|   bjp|          68.23|
|      uttara kannada|   bjp|          68.15|
|          panchmahal|   bjp|          67.56|
|          hazaribagh|   bjp|          67.42|
|         chittorgarh|   bjp|          67.38|
|      ahmedabad east|   bjp|     

In [ ]:
res_df.count()

330

In [ ]:
res_df.write.format("csv")\
.option("header","true")\
.mode("overwrite")\
.save("dbfs:/mnt/analytics/general/consecutive_elected_parties_ranked_by_2019_vote_percentage")

# 4. Which constituencies have voted for different parties in two elections (list top 10
# based on difference (2019-2014) in winnervote percentage in two elections)

In [ ]:
res_df = spark.sql("""
WITH party_2014 AS (
    SELECT
        fact_2014_table.pc_name,
        fact_2014_table.total_votes,
        dim_parties_2014_table.party
    FROM fact_2014_table
    INNER JOIN dim_parties_2014_table
        ON dim_parties_2014_table.party_id = fact_2014_table.party_id
),
total_votes_per_constituency_2014 AS (
    SELECT
        pc_name,
        SUM(total_votes) AS total_votes_per_constituency
    FROM party_2014
    GROUP BY pc_name
),
winning_party_vote_percentage_2014 AS (
    SELECT
        pc_name,
        party,
        total_votes,
        total_votes_per_constituency,
        ROUND((total_votes / total_votes_per_constituency) * 100, 2) AS vote_percentage_2014
    FROM (
        SELECT
            p.pc_name,
            p.party,
            p.total_votes,
            tv.total_votes_per_constituency,
            RANK() OVER (PARTITION BY p.pc_name ORDER BY p.total_votes DESC) AS rank_2014
        FROM party_2014 p
        INNER JOIN total_votes_per_constituency_2014 tv
            ON p.pc_name = tv.pc_name
    ) ranked_2014
    WHERE rank_2014 = 1
),
party_2019 AS (
    SELECT
        fact_2019_table.pc_name,
        fact_2019_table.total_votes,
        dim_parties_2019_table.party
    FROM fact_2019_table
    INNER JOIN dim_parties_2019_table
        ON dim_parties_2019_table.party_id = fact_2019_table.party_id
),
total_votes_per_constituency_2019 AS (
    SELECT
        pc_name,
        SUM(total_votes) AS total_votes_per_constituency
    FROM party_2019
    GROUP BY pc_name
),
winning_party_vote_percentage_2019 AS (
    SELECT
        pc_name,
        party,
        total_votes,
        total_votes_per_constituency,
        ROUND((total_votes / total_votes_per_constituency) * 100, 2) AS vote_percentage_2019
    FROM (
        SELECT
            p.pc_name,
            p.party,
            p.total_votes,
            tv.total_votes_per_constituency,
            RANK() OVER (PARTITION BY p.pc_name ORDER BY p.total_votes DESC) AS rank_2019
        FROM party_2019 p
        INNER JOIN total_votes_per_constituency_2019 tv
            ON p.pc_name = tv.pc_name
    ) ranked_2019
    WHERE rank_2019 = 1
),
different_party_winners AS (
    SELECT
        w2014.pc_name,
        w2014.party AS party_2014,
        w2014.vote_percentage_2014,
        w2019.party AS party_2019,
        w2019.vote_percentage_2019,
        round(w2019.vote_percentage_2019 - w2014.vote_percentage_2014,2) AS diff_winner_percent_votes
    FROM winning_party_vote_percentage_2014 w2014
    INNER JOIN winning_party_vote_percentage_2019 w2019
        ON w2014.pc_name = w2019.pc_name
    WHERE w2014.party != w2019.party
)
SELECT
    pc_name,
    party_2014,
    vote_percentage_2014,
    party_2019,
    vote_percentage_2019,
    diff_winner_percent_votes
FROM different_party_winners
ORDER BY diff_winner_percent_votes DESC
LIMIT 10;


""")


In [ ]:
res_df.show()

+-------------------+----------+--------------------+----------+--------------------+-------------------------+
|            pc_name|party_2014|vote_percentage_2014|party_2019|vote_percentage_2019|diff_winner_percent_votes|
+-------------------+----------+--------------------+----------+--------------------+-------------------------+
|        alipurduars|      aitc|               29.62|       bjp|                54.4|                    24.78|
|autonomous district|       inc|               39.23|       bjp|               61.73|                     22.5|
|      kanniyakumari|       bjp|               37.64|       inc|               59.83|                    22.19|
|         puducherry|     ainrc|               34.57|       inc|               56.27|                     21.7|
|          bhagalpur|       rjd|               37.74|     jd(u)|                59.3|                    21.56|
|            sambhal|       bjp|               34.08|        sp|                55.6|                   

In [ ]:
res_df.write.format("csv")\
.option("header","true")\
.mode("overwrite")\
.save("dbfs:/mnt/analytics/general/top_10_non_consecutive_elected_parties_ranked_by_diff_in_vote_percentage")

# 5. Top 5 candidates based on margin difference with runners in 2014 and 2019.

**in 2014**

In [ ]:
res_2014_df = spark.sql(""" 
with ranking as 
(
select pc_name,candidate_id,total_votes,
row_number() over(partition by pc_name order by total_votes desc) as rnk
from fact_2014_table 
),
top_2 as 
( 
select * 
from ranking 
where rnk <=2
),
derive_winner_runnerup_columns as 
(
select pc_name,candidate_id,total_votes as winner_votes ,rnk,
lead(total_votes,1) over(partition by pc_name order by rnk asc) as runner_up_votes
from top_2
),
derive_margin as 
( 
select pc_name,candidate_id,winner_votes, runner_up_votes,
(winner_votes - runner_up_votes) as margin
from derive_winner_runnerup_columns 
where rnk=1 
order by margin desc 
limit 5
)
select derive_margin.pc_name,dim_candidates_2014_table.candidate,
derive_margin.winner_votes,
derive_margin.runner_up_votes,derive_margin.margin as margin_difference
from derive_margin 
inner join dim_candidates_2014_table 
on derive_margin.candidate_id = dim_candidates_2014_table.candidate_id
order by margin desc
""")

In [ ]:
res_2014_df.show(truncate=False)

+---------+-----------------------+------------+---------------+-----------------+
|pc_name  |candidate              |winner_votes|runner_up_votes|margin_difference|
+---------+-----------------------+------------+---------------+-----------------+
|vadodara |narendra modi          |845464      |275336         |570128           |
|ghaziabad|vijay kumar singh      |758482      |191222         |567260           |
|navsari  |c. r. patil            |820831      |262715         |558116           |
|jaipur   |ramcharan bohara       |863358      |324013         |539345           |
|surat    |darshana vikram jardosh|718412      |185222         |533190           |
+---------+-----------------------+------------+---------------+-----------------+



In [ ]:
res_2014_df.write.format("csv")\
.option("header","true")\
.mode("overwrite")\
.save("dbfs:/mnt/analytics/2014/top_5_candidates_based_on_margin_difference")

**in_2019**

In [ ]:
res_2019_df = spark.sql(""" 
with ranking as 
(
select pc_name,candidate_id,total_votes,
row_number() over(partition by pc_name order by total_votes desc) as rnk
from fact_2019_table 
),
top_2 as 
( 
select * 
from ranking 
where rnk <=2
),
derive_winner_runnerup_columns as 
(
select pc_name,candidate_id,total_votes as winner_votes ,rnk,
lead(total_votes,1) over(partition by pc_name order by rnk asc) as runner_up_votes
from top_2
),
derive_margin as 
( 
select pc_name,candidate_id,winner_votes, runner_up_votes,
(winner_votes - runner_up_votes) as margin
from derive_winner_runnerup_columns 
where rnk=1 
order by margin desc 
limit 5
)
select derive_margin.pc_name,dim_candidates_2019_table.candidate,
derive_margin.winner_votes,
derive_margin.runner_up_votes,derive_margin.margin as margin_difference
from derive_margin 
inner join dim_candidates_2019_table 
on derive_margin.candidate_id = dim_candidates_2019_table.candidate_id
order by margin desc
""")

In [ ]:
res_2019_df.show(truncate=False)

+---------+-----------------------+------------+---------------+-----------------+
|pc_name  |candidate              |winner_votes|runner_up_votes|margin_difference|
+---------+-----------------------+------------+---------------+-----------------+
|navsari  |c. r. patil            |972739      |283071         |689668           |
|karnal   |sanjay bhatia          |911594      |255452         |656142           |
|faridabad|krishan pal            |913222      |274983         |638239           |
|bhilwara |subhash chandra baheria|938160      |326160         |612000           |
|vadodara |ranjanben bhatt        |883719      |294542         |589177           |
+---------+-----------------------+------------+---------------+-----------------+



In [ ]:
res_2019_df.write.format("csv")\
.option("header","true")\
.mode("overwrite")\
.save("dbfs:/mnt/analytics/2019/top_5_candidates_based_on_margin_difference")

# 6. % Split of votes of parties between 2014 vs 2019 at national level

**in 2014**

In [ ]:
res_2014_df=spark.sql(""" 
with cte as 
(
select party_id,sum(total_votes) as total_party_votes 
from fact_2014_table 
group by party_id 
order by total_party_votes desc 
),
cte2 as 
(
select party_id,total_party_votes ,sum(total_party_votes) over() as overall_total_votes
from cte 
),
cte3 as 
(
select party_id,total_party_votes,
round((total_party_votes/overall_total_votes)*100,2) as votes_percentage_split
from cte2 
limit 5
)
select dim_parties_2014_table.party,
 cte3.total_party_votes,cte3.votes_percentage_split 
from cte3 
inner join dim_parties_2014_table 
on cte3.party_id = dim_parties_2014_table.party_id 
order by cte3.votes_percentage_split desc
""")

In [ ]:
res_2014_df.show()

+-----+-----------------+----------------------+
|party|total_party_votes|votes_percentage_split|
+-----+-----------------+----------------------+
|  bjp|        161047761|                 30.97|
|  inc|         96641749|                 18.58|
|  bsp|         22434033|                  4.31|
| aitc|         21215661|                  4.08|
|   sp|         18623266|                  3.58|
+-----+-----------------+----------------------+



In [ ]:
res_2014_df.write.format("csv")\
.option("header","true")\
.mode("overwrite")\
.save("dbfs:/mnt/analytics/2014/top_5_parties_based_on_vote_percentage_split")

**in 2019**

In [ ]:
res_2019_df=spark.sql(""" 
with cte as 
(
select party_id,sum(total_votes) as total_party_votes 
from fact_2019_table 
group by party_id 
order by total_party_votes desc 
),
cte2 as 
(
select party_id,total_party_votes ,sum(total_party_votes) over() as overall_total_votes
from cte 
),
cte3 as 
(
select party_id,total_party_votes,
round((total_party_votes/overall_total_votes)*100,2) as votes_percentage_split
from cte2 
limit 5
)
select dim_parties_2019_table.party,
 cte3.total_party_votes,cte3.votes_percentage_split 
from cte3 
inner join dim_parties_2019_table 
on cte3.party_id = dim_parties_2019_table.party_id 
order by cte3.votes_percentage_split desc
""")

In [ ]:
res_2019_df.show()

+-----+-----------------+----------------------+
|party|total_party_votes|votes_percentage_split|
+-----+-----------------+----------------------+
|  bjp|        229076879|                  37.3|
|  inc|        119495214|                 19.46|
| aitc|         24929330|                  4.06|
|  bsp|         22246501|                  3.62|
|   sp|         15636617|                  2.55|
+-----+-----------------+----------------------+



In [ ]:
res_2019_df.write.format("csv")\
.option("header","true")\
.mode("overwrite")\
.save("dbfs:/mnt/analytics/2019/top_5_parties_based_on_vote_percentage_split")

# 7. % Split of votes of parties between 2014 vs 2019 at state level.

**in 2014**

In [ ]:
res_2014_df = spark.sql("""
with cte as 
(
select state_id,sum(total_votes) as total_statewise_votes
from fact_2014_table
group by state_id  
),
cte2 as 
(
select state_id,party_id,
sum(total_votes) as total_partywise_votes  
from fact_2014_table 
group by state_id,party_id
),
cte3 as 
(
select cte2.*,cte.total_statewise_votes,
round(((cte2.total_partywise_votes/total_statewise_votes)*100),2) as vote_percentage_split
from cte2 
inner join cte 
on cte.state_id = cte2.state_id 
),
cte4 as 
(
select dim_states_table.state_name,cte3.party_id,
cte3.total_partywise_votes,cte3.total_statewise_votes,cte3.vote_percentage_split
from cte3 
inner join dim_states_table 
on dim_states_table.state_id = cte3.state_id
)
select cte4.state_name, dim_parties_2014_table.party,cte4.total_partywise_votes,cte4.total_statewise_votes,
cte4.vote_percentage_split
from cte4 
inner join dim_parties_2014_table 
on cte4.party_id = dim_parties_2014_table.party_id 
order by state_name asc,vote_percentage_split desc
""")

In [ ]:
res_2014_df.filter(col("state_name")=="uttar pradesh").show()

+-------------+-----+---------------------+---------------------+---------------------+
|   state_name|party|total_partywise_votes|total_statewise_votes|vote_percentage_split|
+-------------+-----+---------------------+---------------------+---------------------+
|uttar pradesh|  bjp|             34318854|             81093120|                42.32|
|uttar pradesh|   sp|             17988967|             81093120|                22.18|
|uttar pradesh|  bsp|             15914194|             81093120|                19.62|
|uttar pradesh|  inc|              6061267|             81093120|                 7.47|
|uttar pradesh| aaap|               821302|             81093120|                 1.01|
|uttar pradesh|  rld|               689409|             81093120|                 0.85|
|uttar pradesh| nota|               592331|             81093120|                 0.73|
|uttar pradesh| pecp|               499504|             81093120|                 0.62|
|uttar pradesh|   ad|           

In [ ]:
res_2014_df.show(1)

+--------------------+-----+---------------------+---------------------+---------------------+
|          state_name|party|total_partywise_votes|total_statewise_votes|vote_percentage_split|
+--------------------+-----+---------------------+---------------------+---------------------+
|andaman & nicobar...|  bjp|                90969|               190328|                 47.8|
+--------------------+-----+---------------------+---------------------+---------------------+
only showing top 1 row



In [ ]:
res_2014_df.write.format("csv")\
.option("header","true")\
.mode("overwrite")\
.save("dbfs:/mnt/analytics/2014/vote_percentage_split_on_state_party_level")

**in 2019**

In [ ]:
res_2019_df = spark.sql("""
with cte as 
(
select state_id,sum(total_votes) as total_statewise_votes
from fact_2019_table
group by state_id  
),
cte2 as 
(
select state_id,party_id,
sum(total_votes) as total_partywise_votes  
from fact_2019_table 
group by state_id,party_id
),
cte3 as 
(
select cte2.*,cte.total_statewise_votes,
round(((cte2.total_partywise_votes/total_statewise_votes)*100),2) as vote_percentage_split
from cte2 
inner join cte 
on cte.state_id = cte2.state_id 
),
cte4 as 
(
select dim_states_table.state_name,cte3.party_id,
cte3.total_partywise_votes,cte3.total_statewise_votes,cte3.vote_percentage_split
from cte3 
inner join dim_states_table 
on dim_states_table.state_id = cte3.state_id
)
select cte4.state_name, dim_parties_2019_table.party,cte4.total_partywise_votes,cte4.total_statewise_votes,
cte4.vote_percentage_split
from cte4 
inner join dim_parties_2019_table 
on cte4.party_id = dim_parties_2019_table.party_id 
order by state_name asc,vote_percentage_split desc
""")

In [ ]:
res_2019_df.filter(col("state_name")=="uttar pradesh").show()

+-------------+-------+---------------------+---------------------+---------------------+
|   state_name|  party|total_partywise_votes|total_statewise_votes|vote_percentage_split|
+-------------+-------+---------------------+---------------------+---------------------+
|uttar pradesh|    bjp|             42858171|             86481398|                49.56|
|uttar pradesh|    bsp|             16659754|             86481398|                19.26|
|uttar pradesh|     sp|             15533620|             86481398|                17.96|
|uttar pradesh|    inc|              5457352|             86481398|                 6.31|
|uttar pradesh|    rld|              1447363|             86481398|                 1.67|
|uttar pradesh|   adal|              1039478|             86481398|                  1.2|
|uttar pradesh|   nota|               725097|             86481398|                 0.84|
|uttar pradesh|   sbsp|               277308|             86481398|                 0.32|
|uttar pra

In [ ]:
res_2019_df.write.format("csv")\
.option("header","true")\
.mode("overwrite")\
.save("dbfs:/mnt/analytics/2019/vote_percentage_split_on_state_party_level")

# 8. List top 5 constituencies for two major national parties where they have gained vote
# share in 2019 as compared to 2014.

**drawing inference from question 6 results and general knowledge that 2 major national parties are bjp and inc**

**bjp case**

In [ ]:
spark.sql("""
select * from dim_parties_2014_table 
where party="bjp"        
""").show()

+--------+-----+------------+
|party_id|party|party_symbol|
+--------+-----+------------+
|     134|  bjp|       lotus|
+--------+-----+------------+



In [ ]:
spark.sql("""
select * from dim_parties_2019_table 
where party="bjp"        
""").show()

+-----+------------+--------+
|party|party_symbol|party_id|
+-----+------------+--------+
|  bjp|       lotus|     180|
+-----+------------+--------+



In [ ]:
bjp_vote_share_df = spark.sql(""" 
with q1 as 
(
select pc_name,total_votes as bjp_total_votes
from fact_2014_table 
where party_id = 134   
),
q2 as 
(
select pc_name,sum(total_votes) as total_constituency_wise_votes
from fact_2014_table 
group by  pc_name
),
q3 as
(
select q2.pc_name,
round((q1.bjp_total_votes/q2.total_constituency_wise_votes)*100,2) as vote_percentage_2014
from q2
inner join q1 
on q2.pc_name = q1.pc_name
),
q4 as 
(
select pc_name,total_votes as bjp_total_votes
from fact_2019_table 
where party_id = 180   
),
q5 as 
(
select pc_name,sum(total_votes) as total_constituency_wise_votes
from fact_2019_table 
group by  pc_name
),
q6 as
(
select q4.pc_name,
round((q4.bjp_total_votes/q5.total_constituency_wise_votes)*100,2) as vote_percentage_2019
from q4
inner join q5
on q4.pc_name = q5.pc_name
),
q7 as
(
select q6.pc_name,q3.vote_percentage_2014,q6.vote_percentage_2019
from q6 
inner join q3 
on q6.pc_name = q3.pc_name
),
vote_percentage_change as 
(
select *,
round((vote_percentage_2019-vote_percentage_2014),2) as change_in_vote_share_percentage
from q7
)
select * from vote_percentage_change
""")

In [ ]:
top_5_bjp_vote_share_gain_df = bjp_vote_share_df.orderBy(col("change_in_vote_share_percentage").desc()).limit(5) 



In [ ]:
top_5_bjp_vote_share_gain_df.show() 

+------------+--------------------+--------------------+-------------------------------+
|     pc_name|vote_percentage_2014|vote_percentage_2019|change_in_vote_share_percentage|
+------------+--------------------+--------------------+-------------------------------+
|tripura west|                 5.1|               51.77|                          46.67|
|     purulia|                7.16|                49.3|                          42.14|
|tripura east|                6.37|               46.12|                          39.75|
|    ranaghat|               17.27|               52.78|                          35.51|
|    jhargram|                9.74|               44.56|                          34.82|
+------------+--------------------+--------------------+-------------------------------+



In [ ]:
top_5_bjp_vote_share_gain_df.write.format("csv")\
.option("header","true")\
.mode("overwrite")\
.save("dbfs:/mnt/analytics/general/top_5_constituencies_where_bjp_has_vote_share_gain")

**inc case**

In [ ]:
spark.sql("""
select * from dim_parties_2014_table 
where party="inc"        
""").show()

+--------+-----+------------+
|party_id|party|party_symbol|
+--------+-----+------------+
|     258|  inc|        hand|
+--------+-----+------------+



In [ ]:
spark.sql("""
select * from dim_parties_2019_table 
where party="inc"        
""").show()

+-----+------------+--------+
|party|party_symbol|party_id|
+-----+------------+--------+
|  inc|        hand|     332|
+-----+------------+--------+



In [ ]:
inc_vote_share_df = spark.sql(""" 
with q1 as 
(
select pc_name,total_votes as inc_total_votes
from fact_2014_table 
where party_id = 258   
),
q2 as 
(
select pc_name,sum(total_votes) as total_constituency_wise_votes
from fact_2019_table 
group by  pc_name
),
q3 as
(
select q2.pc_name,
round((q1.inc_total_votes/q2.total_constituency_wise_votes)*100,2) as vote_percentage_2014
from q2
inner join q1 
on q2.pc_name = q1.pc_name
),
q4 as 
(
select pc_name,total_votes as inc_total_votes
from fact_2019_table 
where party_id = 332   
),
q5 as 
(
select pc_name,sum(total_votes) as total_constituency_wise_votes
from fact_2019_table 
group by  pc_name
),
q6 as
(
select q4.pc_name,
round((q4.inc_total_votes/q5.total_constituency_wise_votes)*100,2) as vote_percentage_2019
from q4
inner join q5
on q4.pc_name = q5.pc_name
),
q7 as
(
select q6.pc_name,q3.vote_percentage_2014,q6.vote_percentage_2019
from q6 
inner join q3 
on q6.pc_name = q3.pc_name
),
vote_percentage_change as 
(
select *,
round((vote_percentage_2019-vote_percentage_2014),2) as change_in_vote_share_percentage
from q7
)
select * from vote_percentage_change
""")

In [ ]:
top_5_inc_vote_share_gain_df = inc_vote_share_df.orderBy(col("change_in_vote_share_percentage").desc()).limit(5)

In [ ]:
top_5_inc_vote_share_gain_df.show()

+---------------+--------------------+--------------------+-------------------------------+
|        pc_name|vote_percentage_2014|vote_percentage_2019|change_in_vote_share_percentage|
+---------------+--------------------+--------------------+-------------------------------+
|          karur|                2.76|               63.06|                           60.3|
|tiruchirappalli|                4.92|               59.28|                          54.36|
|          arani|                2.42|                54.0|                          51.58|
|    thiruvallur|                3.12|               54.49|                          51.37|
|    krishnagiri|                3.35|               52.64|                          49.29|
+---------------+--------------------+--------------------+-------------------------------+



In [ ]:
top_5_inc_vote_share_gain_df.write.format("csv")\
.option("header","true")\
.mode("overwrite")\
.save("dbfs:/mnt/analytics/general/top_5_constituencies_where_inc_has_vote_share_gain")

# 9. List top 5 constituencies for two major national parties where they have lost vote
# share in 2019 as compared to 2014.

**bjp case**

In [ ]:
top_5_bjp_vote_share_loss_df = bjp_vote_share_df.orderBy(col("change_in_vote_share_percentage").asc()).limit(5) 

In [ ]:
top_5_bjp_vote_share_loss_df.show()

+-------------+--------------------+--------------------+-------------------------------+
|      pc_name|vote_percentage_2014|vote_percentage_2019|change_in_vote_share_percentage|
+-------------+--------------------+--------------------+-------------------------------+
|   narsapuram|               49.62|                1.05|                         -48.57|
|visakhapatnam|               48.72|                2.73|                         -45.99|
|     tirupati|               44.76|                1.23|                         -43.53|
|       sangli|               58.43|               42.77|                         -15.66|
|  daman & diu|               53.83|               42.98|                         -10.85|
+-------------+--------------------+--------------------+-------------------------------+



In [ ]:
top_5_bjp_vote_share_loss_df.write.format("csv")\
.option("header","true")\
.mode("overwrite")\
.save("dbfs:/mnt/analytics/general/top_5_constituencies_where_bjp_has_vote_share_loss")

**inc case**

In [ ]:
top_5_inc_vote_share_loss_df = inc_vote_share_df.orderBy(col("change_in_vote_share_percentage").asc()).limit(5)

In [ ]:
top_5_inc_vote_share_loss_df.show()

+--------------------+--------------------+--------------------+-------------------------------+
|             pc_name|vote_percentage_2014|vote_percentage_2019|change_in_vote_share_percentage|
+--------------------+--------------------+--------------------+-------------------------------+
|ratnagiri - sindh...|               38.07|                7.02|                         -31.05|
|      arunachal west|               35.62|               14.22|                          -21.4|
|           nizamabad|               25.61|                6.52|                         -19.09|
|             raiganj|               24.73|                6.55|                         -18.18|
|       outer manipur|               34.59|               17.77|                         -16.82|
+--------------------+--------------------+--------------------+-------------------------------+



In [ ]:
top_5_inc_vote_share_loss_df.write.format("csv")\
.option("header","true")\
.mode("overwrite")\
.save("dbfs:/mnt/analytics/general/top_5_constituencies_where_inc_has_vote_share_loss")

# 10.Which constituency has voted the most for NOTA?

**2014 case**

In [ ]:
spark.sql("""
select *
from dim_candidates_2014_table 
where candidate="nota"
""").show()

+------------+---------+---+---+--------+
|candidate_id|candidate|sex|age|category|
+------------+---------+---+---+--------+
|        4617|     nota| na| -1|      na|
+------------+---------+---+---+--------+



In [ ]:
top_5_constituencies_voting_nota_2014 = spark.sql("""
with only_nota_data as 
(
select pc_name,total_votes as nota_total_votes
from fact_2014_table
where candidate_id = 4617
),
pc_wise_votes as 
( 
select pc_name,sum(total_votes) as constituency_wise_total_votes
from fact_2014_table 
group by pc_name
) 
select only_nota_data.pc_name,
round((only_nota_data.nota_total_votes/pc_wise_votes.constituency_wise_total_votes)*100,2) as nota_vote_percentage
from only_nota_data 
inner join pc_wise_votes 
on only_nota_data.pc_name = pc_wise_votes.pc_name 
order by nota_vote_percentage desc 
limit 5
""")

In [ ]:
top_5_constituencies_voting_nota_2014.show()

+---------------+--------------------+
|        pc_name|nota_vote_percentage|
+---------------+--------------------+
|       nilgiris|                4.99|
|           tura|                4.19|
|          dahod|                3.59|
|      singhbhum|                 3.4|
|samastipur (sc)|                3.38|
+---------------+--------------------+



In [ ]:
top_5_constituencies_voting_nota_2014.write.format("csv")\
.option("header","true")\
.mode("overwrite")\
.save("dbfs:/mnt/analytics/2014/top_5_constituencies_voting_nota")

**2019 case**

In [ ]:
spark.sql("""
select *
from dim_candidates_2019_table 
where candidate="nota"
""").show()

+------------+---------+---+---+--------+
|candidate_id|candidate|sex|age|category|
+------------+---------+---+---+--------+
|        4721|     nota| na| -1|      na|
+------------+---------+---+---+--------+



In [ ]:
top_5_constituencies_voting_nota_2019 = spark.sql("""
with only_nota_data as 
(
select pc_name,total_votes as nota_total_votes
from fact_2019_table
where candidate_id = 4721
),
pc_wise_votes as 
( 
select pc_name,sum(total_votes) as constituency_wise_total_votes
from fact_2019_table 
group by pc_name
) 
select only_nota_data.pc_name,
round((only_nota_data.nota_total_votes/pc_wise_votes.constituency_wise_total_votes)*100,2) as nota_vote_percentage
from only_nota_data 
inner join pc_wise_votes 
on only_nota_data.pc_name = pc_wise_votes.pc_name 
order by nota_vote_percentage desc 
limit 5
""")

In [ ]:
top_5_constituencies_voting_nota_2019.show()

+-----------------+--------------------+
|          pc_name|nota_vote_percentage|
+-----------------+--------------------+
|   gopalganj (sc)|                5.04|
|           bastar|                4.56|
|paschim champaran|                4.51|
|            aruku|                4.46|
|       jamui (sc)|                4.16|
+-----------------+--------------------+



In [ ]:
top_5_constituencies_voting_nota_2019.write.format("csv")\
.option("header","true")\
.mode("overwrite")\
.save("dbfs:/mnt/analytics/2019/top_5_constituencies_voting_nota")

# 11.Which constituencies have elected candidates whose party has less than 10% vote
# share at state level in 2019?

In [ ]:
res_df = spark.sql("""
with derive_party_votes as 
(
select state_id,party_id,sum(total_votes) as total_party_votes
from  fact_2019_table 
group by state_id,party_id

),
derive_state_votes as 
(
select state_id,sum(total_votes) as total_state_votes
from  fact_2019_table 
group by state_id
),
derive_vote_percentage_of_party_in_state as 
(
select derive_state_votes.state_id,
derive_party_votes.party_id,
round((derive_party_votes.total_party_votes/derive_state_votes.total_state_votes)*100,2)
as vote_percentage_of_party_in_state
from derive_party_votes 
inner join derive_state_votes 
on derive_party_votes.state_id = derive_state_votes.state_id
),
derive_winning_candidates as 
(
select state_id,pc_name,party_id,candidate_id,total_votes,
row_number() over(partition by pc_name order by total_votes desc) as rn
from fact_2019_table 
),
winning_candidates as 
(
select state_id,pc_name,party_id,candidate_id,total_votes
from derive_winning_candidates 
where rn=1
),
join_vote_percentage_of_party_to_winning_candidates as 
(
select winning_candidates.*,derive_vote_percentage_of_party_in_state.vote_percentage_of_party_in_state 
from winning_candidates 
inner join derive_vote_percentage_of_party_in_state 
on winning_candidates.state_id = derive_vote_percentage_of_party_in_state.state_id and 
winning_candidates.party_id = derive_vote_percentage_of_party_in_state.party_id
),
parties_with_less_than_10_percent_vote_share as 
(
select * 
from join_vote_percentage_of_party_to_winning_candidates 
where vote_percentage_of_party_in_state < 10 
),
actual_party_names as 
(
select parties_with_less_than_10_percent_vote_share.state_id,
parties_with_less_than_10_percent_vote_share.pc_name,
parties_with_less_than_10_percent_vote_share.candidate_id,
dim_parties_2019_table.party,
parties_with_less_than_10_percent_vote_share.vote_percentage_of_party_in_state
from parties_with_less_than_10_percent_vote_share 
inner join dim_parties_2019_table 
on dim_parties_2019_table.party_id = parties_with_less_than_10_percent_vote_share.party_id
),
actual_candidate_names as 
(
select actual_party_names.state_id,
actual_party_names.pc_name,dim_candidates_2019_table.candidate,
actual_party_names.party,actual_party_names.vote_percentage_of_party_in_state 
from actual_party_names 
inner join dim_candidates_2019_table 
on actual_party_names.candidate_id = dim_candidates_2019_table.candidate_id
),
actual_state_names as  
(
select dim_states_table.state_name,actual_candidate_names.pc_name,
actual_candidate_names.candidate,actual_candidate_names.party,
actual_candidate_names.vote_percentage_of_party_in_state
from actual_candidate_names 
inner join dim_states_table 
on actual_candidate_names.state_id = dim_states_table.state_id
),
final_output as 
(
select * from actual_state_names 
)
select * from final_output 
order by vote_percentage_of_party_in_state asc
""")

In [ ]:
res_df.show()

+-------------+--------------------+--------------------+------+---------------------------------+
|   state_name|             pc_name|           candidate| party|vote_percentage_of_party_in_state|
+-------------+--------------------+--------------------+------+---------------------------------+
|  maharashtra|mumbai south central|rahul ramesh shewale|   shs|                             0.79|
|  maharashtra|            amravati|    navnit ravi rana|   ind|                             0.95|
|   tamil nadu|      ramanathapuram|        k. navaskani|  iuml|                             1.08|
|   tamil nadu|         chidambaram|thirumaavalavan thol|   vck|                             1.15|
|uttar pradesh|         robertsganj|     pakauri lal kol|  adal|                              1.2|
|uttar pradesh|            mirzapur|anupriya singh patel|  adal|                              1.2|
|    rajasthan|              nagaur|     hanuman beniwal|  rltp|                             2.03|
|    karna

In [ ]:
res_df.count()

37

In [ ]:
res_df.write.format("csv")\
.option("header","true")\
.mode("overwrite")\
.save("dbfs:/mnt/analytics/2019/winning_candidates_whose_parties_have_less_than_10_percent_share_at_state_level")

## Correlation refers to the statistical relationship between the two entities. It measures the extent to which two variables are linearly related. For example, the height and weight of a person are related, and taller people tend to be heavier than shorter people

## The correlation coefficient is usually represented using the symbol r, and it ranges from -1 to +1.
## 
## A correlation coefficient quite close to 0, but either positive or negative, implies little or no relationship between the two variables. A correlation coefficient close to plus 1 means a positive relationship between the two variables, with increases in one of the variables being associated with increases in the other variable.
## 
## A correlation coefficient close to -1 indicates a negative relationship between two variables, with an increase in one of the variables being associated with a decrease in the other variable.

# Secondary Analysis

# Is there a correlation between postalvotes % and voter turnout %?

In [ ]:
df_2019 = spark.sql("""
with state_wise as 
(
select state_id,
sum(total_votes) as state_wise_total_votes,
sum(postal_votes) as state_wise_postal_votes,
sum(total_electors) as state_wise_total_electors
from fact_2019_table 
group by state_id
), 
compute_turn_out_percentages as 
(
select state_id,
round((state_wise_total_votes/state_wise_total_electors)*100,2) as voter_turnout_percentage,
round((state_wise_postal_votes/state_wise_total_electors)*100,2) as postal_turnout_percentage
from state_wise
)
select dim_states_table.state_name,
compute_turn_out_percentages.voter_turnout_percentage,
compute_turn_out_percentages.postal_turnout_percentage
from compute_turn_out_percentages
inner join dim_states_table 
on dim_states_table.state_id = compute_turn_out_percentages.state_id
""")

In [ ]:
df_2019.show(36)

+--------------------+------------------------+-------------------------+
|          state_name|voter_turnout_percentage|postal_turnout_percentage|
+--------------------+------------------------+-------------------------+
|          tamil nadu|                    3.18|                     0.02|
|       uttar pradesh|                    4.47|                     0.01|
|              punjab|                    2.94|                     0.01|
|              odisha|                    7.85|                     0.02|
|          puducherry|                    4.28|                      0.0|
|             haryana|                    3.02|                     0.01|
|           meghalaya|                   12.15|                     0.02|
|andaman & nicobar...|                    4.07|                      0.0|
|    himachal pradesh|                    5.89|                     0.07|
|           karnataka|                    3.71|                     0.01|
|          chandigarh|                

In [ ]:
df_2019.corr("voter_turnout_percentage", "postal_turnout_percentage")

0.32526239067752943

**this shows a very weak postive relationship or no relation between voter turnnout and postal percentage in 2019 data**

In [ ]:
df_2019.write.format("csv")\
.option("header","true")\
.mode("overwrite")\
.save("dbfs:/mnt/analytics/secondary-analysis/correlation_between_voter_turnout_percentage_and_postal_percentage")

# Is there any correlation between GDP of a state and voter turnout %??

# 2018-2019 state wise gdp data 
# source :- https://en.wikipedia.org/wiki/List_of_Indian_states_and_union_territories_by_GDP

In [ ]:
gdp_data = [
    ("andaman & nicobar islands",90030),
    ("andhra pradesh",8737210),
    ("arunachal pradesh",253350),
    ("assam",3093360),
    ("bihar",5279760),
    ("chandigarh",401390),
    ("chhattisgarh",3271070),
    ("dadra & nagar haveli",0),
    ("daman & diu",0),
    ("goa",718530),
    ("gujarat",14921560),
    ("haryana",6989400),
    ("himachal pradesh",1483830),
    ("jammu & kashmir",1598590),
    ("jharkhand",3056950),
    ("karnataka",14793910),
    ("kerala",7882860),
    ("lakshadweep",0),
    ("madhya pradesh",8298050),
    ("maharashtra",25288540),
    ("manipur",273880),
    ("meghalaya",321760),
    ("nagaland",265270),
    ("nct of delhi",6779000),
    ("odisha",4403950),
    ("puducherry",321290),
    ("punjab",4710140),
    ("rajasthan",8325290),
    ("sikkim",259710),
    ("tamil nadu",14650510), 
    ("telangana",7500500),
    ("tripura",437160), 
    ("uttar pradesh",14399260),
    ("uttarakhand",2202220),
    ("west bengal",9747000)

]

In [ ]:
my_schema = StructType([
    StructField("state_name",StringType()),
    StructField("gdp_in_millions_of_indian_rupees",LongType())
]) 

gdp_df = spark.createDataFrame(data=gdp_data,schema=my_schema) 

In [ ]:
gdp_df.show(2)

+--------------------+--------------------------------+
|          state_name|gdp_in_millions_of_indian_rupees|
+--------------------+--------------------------------+
|andaman & nicobar...|                           90030|
|      andhra pradesh|                         8737210|
+--------------------+--------------------------------+
only showing top 2 rows



In [ ]:
df = df_2019.join(gdp_df,df_2019["state_name"]==gdp_df["state_name"],"inner")\
.select(df_2019["state_name"],df_2019["voter_turnout_percentage"],gdp_df["gdp_in_millions_of_indian_rupees"])


In [ ]:
df.show(2)

+--------------------+------------------------+--------------------------------+
|          state_name|voter_turnout_percentage|gdp_in_millions_of_indian_rupees|
+--------------------+------------------------+--------------------------------+
|andaman & nicobar...|                    4.07|                           90030|
|      andhra pradesh|                    5.81|                         8737210|
+--------------------+------------------------+--------------------------------+
only showing top 2 rows



In [ ]:
df.corr("voter_turnout_percentage", "gdp_in_millions_of_indian_rupees")

-0.4458637187518575

**this shows a very weak negative relationship or no relation between voter turnnout state and gdp of state**

In [ ]:
df.write.format("csv")\
.option("header","true")\
.mode("overwrite")\
.save("dbfs:/mnt/analytics/secondary-analysis/correlation_between_voter_turnout_percentage_and_gdp_of_state")

# Is there any correlation between literacy % of a state and voter turnout %?

**Source :- https://en.wikipedia.org/wiki/List_of_Indian_states_and_union_territories_by_literacy_rate**

In [ ]:
literacy_data = [
    ("andaman & nicobar islands",86.27),
    ("andhra pradesh",66.9),
    ("arunachal pradesh",66.95),
    ("assam",85.9),
    ("bihar",70.9),
    ("chandigarh",86.43),
    ("chhattisgarh",77.3),
    ("dadra & nagar haveli",77.65),
    ("daman & diu",87.07),
    ("goa",87.4),
    ("gujarat",82.4),
    ("haryana",80.4),
    ("himachal pradesh",86.6),
    ("jammu & kashmir",77.3),
    ("jharkhand",74.3),
    ("karnataka",77.2),
    ("kerala",96.2),
    ("lakshadweep",92.28),
    ("madhya pradesh",73.7),
    ("maharashtra",84.8),
    ("manipur",79.85),
    ("meghalaya",75.48),
    ("nagaland",80.11),
    ("nct of delhi",88.70),
    ("odisha",77.3),
    ("puducherry",86.55),
    ("punjab",83.7),
    ("rajasthan",69.7),
    ("sikkim",82.2),
    ("tamil nadu",82.9), 
    ("telangana",72.8),
    ("tripura",87.75), 
    ("uttar pradesh",73.0),
    ("uttarakhand",87.6),
    ("west bengal",80.50)

]

In [ ]:
my_schema = StructType([
    StructField("state_name",StringType()),
    StructField("literacy_rate",FloatType()),
]) 

literacy_df = spark.createDataFrame(data=literacy_data,schema=my_schema) 

In [ ]:
literacy_df.show(2)

+--------------------+-------------+
|          state_name|literacy_rate|
+--------------------+-------------+
|andaman & nicobar...|        86.27|
|      andhra pradesh|         66.9|
+--------------------+-------------+
only showing top 2 rows



In [ ]:
df = df_2019.join(literacy_df,df_2019["state_name"]==literacy_df["state_name"],"inner")\
.select(df_2019["state_name"],df_2019["voter_turnout_percentage"],literacy_df["literacy_rate"])

In [ ]:
df.show(2)

+--------------------+------------------------+-------------+
|          state_name|voter_turnout_percentage|literacy_rate|
+--------------------+------------------------+-------------+
|andaman & nicobar...|                    4.07|        86.27|
|      andhra pradesh|                    5.81|         66.9|
+--------------------+------------------------+-------------+
only showing top 2 rows



In [ ]:
df.corr("voter_turnout_percentage", "literacy_rate")

0.07024954230941735

**This shows there is no reationship between voter turn out percentage and literacy rate**

In [ ]:
df.write.format("csv")\
.option("header","true")\
.mode("overwrite")\
.save("dbfs:/mnt/analytics/secondary-analysis/correlation_between_voter_turnout_percentage_and_literacy_rate_of_state")